# Sentiment Analysis using Doc2Vec

## Setup

### Modules

In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import logging
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

### Input Format

- `test-neg.txt`: 12500 negative movie reviews from the test data
- `test-pos.txt`: 12500 positive movie reviews from the test data
- `train-neg.txt`: 12500 negative movie reviews from the training data
- `train-pos.txt`: 12500 positive movie reviews from the training data
- `train-unsup.txt`: 50000 Unlabelled movie reviews

## Model

### Building the Vocabulary Table

Doc2Vec requires us to build the vocabulary table (simply digesting all the words and filtering out the unique words, and doing some basic counts on them). So we feed it the array of sentences. `model.build_vocab` takes an array of `LabeledLineSentence`, hence our `to_array` function in the `LabeledLineSentences` class. 

If you're curious about the parameters, do read the Word2Vec documentation. Otherwise, here's a quick rundown:

- `min_count`: ignore all words with total frequency lower than this. You have to set this to 1, since the sentence labels only appear once. Setting it any higher than 1 will miss out on the sentences.
- `window`: the maximum distance between the current and predicted word within a sentence. Word2Vec uses a skip-gram model, and this is simply the window size of the skip-gram model.
- `size`: dimensionality of the feature vectors in output. 100 is a good number. If you're extreme, you can go up to around 400.
- `sample`: threshold for configuring which higher-frequency words are randomly downsampled
- `workers`: use this many worker threads to train the model 

In [4]:
model = Doc2Vec(min_count=1, window=10, size=100, sample=1e-4, negative=5, workers=7)

model.build_vocab(sentences.to_array())

### Training Doc2Vec

Now we train the model. The model is better trained if **in each training epoch, the sequence of sentences fed to the model is randomized**. This is important: missing out on this steps gives you really shitty results. This is the reason for the `sentences_perm` method in our `LabeledLineSentences` class.

We train it for 10 epochs. If I had more time, I'd have done 20.

This process takes around 10 mins, so go grab some coffee.

In [ ]:
for epoch in range(10):
    model.train(sentences.sentences_perm())

### Inspecting the Model

Let's see what our model gives. It seems that it has kind of understood the word `good`, since the most similar words to good are `glamorous`, `spectacular`, `astounding` etc. This is really awesome (and important), since we are doing sentiment analysis.

In [46]:
model.most_similar('good')

[(u'tekashi', 0.45127424597740173),
 (u'glamorous', 0.4344240427017212),
 (u'spectacular', 0.42718690633773804),
 (u'astounding', 0.42001062631607056),
 (u'valentinov', 0.41705751419067383),
 (u'sweetest', 0.4043062925338745),
 (u'complementary', 0.4039931297302246),
 (u'boyyyyy', 0.39713743329048157),
 (u'macdonaldsland', 0.3965899348258972),
 (u'elven', 0.39042729139328003)]

We can also prop the hood open and see what the model actually contains. This is each of the vectors of the words and sentences in the model. We can access all of them using `model.syn0` (for the geekier ones among you, `syn0` is simply the output layer of the shallow neural network). However, we don't want to use the entire `syn0` since that contains the vectors for the words as well, but we are only interested in the ones for sentences.

Here's a sample vector for the first sentence in the training set for negative reviews:

In [47]:
model['TRAIN_NEG_0']

array([ 0.45238438, -0.07346677, -0.17444436,  0.60655016, -0.70522565,
        0.28476399,  0.24404588,  0.09271102, -0.02715847, -0.13526627,
       -0.12390804, -0.00219905,  0.011253  ,  0.24557671, -0.09958933,
        0.17554867,  0.16079453, -0.18499082, -0.31598854,  0.01447532,
        0.52194822, -0.2387463 ,  0.16799606,  0.47053325,  0.09696233,
       -0.2582404 , -0.19224562, -0.07114315, -0.25864932, -0.5387702 ,
        0.01053433,  0.43367237,  0.07885301,  0.04634216,  0.0899957 ,
        0.06260718, -0.38053334,  0.18118465,  0.14301547,  0.18286002,
       -0.31105465,  0.2040111 , -0.76622951,  0.06977512,  0.11759907,
       -0.11566088, -0.00373716, -0.14705311, -0.29019266, -0.04825564,
        0.20127594, -0.0258627 , -0.20973501,  0.48925173, -0.31426486,
        0.3180953 ,  0.41300809, -0.29024398, -0.21187432,  0.10730035,
        0.30392009, -0.2130826 ,  0.47062019, -0.17570473,  0.21256927,
        0.51417089, -0.00951673,  0.1525774 ,  0.05895659,  0.33

### Saving and Loading Models

To avoid training the model again, we can save it.

In [48]:
model.save('./imdb.d2v')

And load it.

In [184]:
model = Doc2Vec.load('./imdb.d2v')

## Classifying Sentiments

### Training Vectors

Now let's use these vectors to train a classifier. First, we must extract the training vectors. Remember that we have a total of 25000 training reviews, with equal numbers of positive and negative ones (12500 positive, 12500 negative).

Hence, we create a `numpy` array (since the classifier we use only takes numpy arrays. There are two parallel arrays, one containing the vectors (`train_arrays`) and the other containing the labels (`train_labels`).

We simply put the positive ones at the first half of the array, and the negative ones at the second half.

In [49]:
train_arrays = numpy.zeros((25000, 100))
train_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    train_arrays[i] = model[prefix_train_pos]
    train_arrays[12500 + i] = model[prefix_train_neg]
    train_labels[i] = 1
    train_labels[12500 + i] = 0

The training array looks like this: rows and rows of vectors representing each sentence.

In [50]:
print train_arrays

[[ 0.42028627 -0.0910796  -0.10316094 ..., -0.11574443  0.54547763
  -0.1086079 ]
 [ 0.21860494  0.34468749 -0.06821636 ..., -0.02118306  0.39692196
   0.518085  ]
 [ 0.19905667 -0.05517581  0.0789782  ...,  0.78548694  0.10369277
   0.15604787]
 ..., 
 [ 0.42894334 -0.03023763 -0.38231012 ...,  0.17735066  0.36474037
  -0.08756389]
 [ 0.65340477  0.388024   -0.34454256 ...,  0.0466847   0.61409295
   0.19534792]
 [ 0.40329584  0.26531416 -0.11242788 ...,  0.08738184  0.48685795
  -0.17476116]]


The labels are simply category labels for the sentence vectors -- 1 representing positive and 0 for negative.

In [58]:
print train_labels

[ 1.  1.  1. ...,  0.  0.  0.]


### Testing Vectors

We do the same for testing data -- data that we are going to feed to the classifier after we've trained it using the training data. This allows us to evaluate our results. The process is pretty much the same as extracting the results for the training data.

In [59]:
test_arrays = numpy.zeros((25000, 100))
test_labels = numpy.zeros(25000)

for i in range(12500):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    test_arrays[i] = model[prefix_test_pos]
    test_arrays[12500 + i] = model[prefix_test_neg]
    test_labels[i] = 1
    test_labels[12500 + i] = 0

### Classification

Now we train a logistic regression classifier using the training data.

In [51]:
classifier = LogisticRegression()
classifier.fit(train_arrays, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

And find that we have achieved near 87% accuracy for sentiment analysis. This is rather incredible, given that we are only using a linear SVM and a very shallow neural network.

In [55]:
classifier.score(test_arrays, test_labels)

0.86968000000000001

Isn't this fantastic? Hope I saved you some time!

## References

- Doc2vec: https://radimrehurek.com/gensim/models/doc2vec.html
- Paper that inspired this: http://arxiv.org/abs/1405.4053